# Setup
Installing needed libraries

In [1]:
! pip install numpy==1.26.4 scikit-learn==1.2.2 fastprogress==1.0.3 transformers==4.38.1 pandas==2.1.4 tqdm==4.66.1 torch==2.1.2 torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 78.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 83.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 76.6 MB/s eta 0:00:00:00:01
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.4
    Uninstalling tqdm-4.66.4:
      Successfully uninstalled tqdm-4.66.4
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transfo

In [2]:
# optional
! pip install git+https://github.com/Haruray/ekphrasis-custom.git

  Cloning https://github.com/Haruray/ekphrasis-custom.git to /tmp/pip-req-build-_koevexq
  Running command git clone --filter=blob:none --quiet https://github.com/Haruray/ekphrasis-custom.git /tmp/pip-req-build-_koevexq
  Resolved https://github.com/Haruray/ekphrasis-custom.git to commit 33f15ce8eef096efe40b134aac0c06f437cd9a03
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 1.9 MB/s eta 0:00:00
  Created wheel for ekphrasis: filename=ekphrasis-0.5.52-py3-none-any.whl size=617095 sha256=8369c674a0d03933a1f871f758c509c0ca7d3eb837ebeca444a180a1b4d6f0bc
  Stored in directory: /tmp/pip-ephem-wheel-cache-lwea23m2/wheels/db/aa/78/3324febc0249d71447c15e7cdf65517757e106402e6d6bb448
  Created wheel for mtranslate: filename=mtranslate-1.8-py3-none-any.whl size=3672 sha256=a57553a1d8c8e3b65649053ec8ab91745552395ae527693c879d4a08fe

In [3]:
import os

## Cloning Source Code Repository

In [4]:
%cd /kaggle/working/
if not os.path.isdir("TA-Multi-Label-Emotion-Classification-Indonesian-Text"):
  print("Cloning repo...")
  !git clone https://github.com/Haruray/TA-Multi-Label-Emotion-Classification-Indonesian-Text
%cd /kaggle/working/TA-Multi-Label-Emotion-Classification-Indonesian-Text
!git pull https://github.com/Haruray/TA-Multi-Label-Emotion-Classification-Indonesian-Text
    
%cd /kaggle/working/TA-Multi-Label-Emotion-Classification-Indonesian-Text/src

/kaggle/working
Cloning repo...
Cloning into 'TA-Multi-Label-Emotion-Classification-Indonesian-Text'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 52 (delta 13), reused 52 (delta 13), pack-reused 0
Unpacking objects: 100% (52/52), 199.51 KiB | 6.04 MiB/s, done.
/kaggle/working/TA-Multi-Label-Emotion-Classification-Indonesian-Text
From https://github.com/Haruray/TA-Multi-Label-Emotion-Classification-Indonesian-Text
 * branch            HEAD       -> FETCH_HEAD
Already up to date.
/kaggle/working/TA-Multi-Label-Emotion-Classification-Indonesian-Text/src


In [5]:
from MLEC import Trainer, SpanEmo, DataClass, EmoRec, EarlyStopping, EvaluateOnTest
from torch.utils.data import DataLoader
import torch
import datetime
import json
import numpy as np
import wandb
import shutil

In [6]:
seed = 42
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Currently using {}".format(device))

def set_seed(seed):    
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed(seed)

Currently using cuda:0


In [7]:
now = datetime.datetime.now()
filename = now.strftime("%Y-%m-%d %H-%M-%S")
# fw = open("../configs/" + filename + ".json", "a")
model_path = filename + ".pt"

## Constants

In [8]:
CHECKPOINT_PATH = model_path
MODEL_PATH = "/kaggle/working/TA-Multi-Label-Emotion-Classification-Indonesian-Text/models/"+filename+"_checkpoint.pt"
TRAIN_FILEPATH = "/kaggle/working/TA-Multi-Label-Emotion-Classification-Indonesian-Text/data/id_df_train_new.csv"
DEV_FILEPATH = "/kaggle/working/TA-Multi-Label-Emotion-Classification-Indonesian-Text/data/id_df_dev_new.csv"
TEST_FILEPATH = "/kaggle/working/TA-Multi-Label-Emotion-Classification-Indonesian-Text/data/id_df_test_new.csv"
MAX_LENGTH = 128
DROPOUT_RATE = 0.1
TOKENIZER_NAME = "indolem/indobert-base-uncased"
ENCODER_NAME = "indolem/indobert-base-uncased"

# === SPANEMO PARAMS ===
CLASSIFIER_LEARNING_RATE = 0.005
ENCODER_LEARNING_RATE = 2e-5
TRAIN_BATCH_SIZE = 16
EVAL_BATCH_SIZE = 16
LCA_LOSSES = [0.6]
ZLPR_LOSSES = [0.4]

# === INDOBERT PARAMS ===
# CLASSIFIER_LEARNING_RATE = 5e-5
# ENCODER_LEARNING_RATE = 5e-5
# TRAIN_BATCH_SIZE = 32
# EVAL_BATCH_SIZE = 32
# LCA_LOSSES = [0.9]
# ZLPR_LOSSES = [0.0]

MAX_EPOCH = 20
PATIENCE = 20
MODEL_TO_TRAIN = "spanemo" #  "emorec" or "spanemo"
USE_PREFIX = MODEL_TO_TRAIN == "spanemo" # only True if its SpanEmo
PROJECT_NAME = "TA Revisi" # wandb project name

LCA_LOSSES = [0.0, 0.6]
ZLPR_LOSSES = [0.0, 0.4]

os.environ["WANDB_API_KEY"] = "" # enter wandb api key here

# Dataset Processing

In [9]:
train_dataset = DataClass(
    filename=TRAIN_FILEPATH,
    max_length=MAX_LENGTH,
    tokenizer_name=TOKENIZER_NAME,
    use_prefix=USE_PREFIX
)
train_data_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)

print("The number of training batches: ", len(train_data_loader))

dev_dataset = DataClass(
    filename=DEV_FILEPATH,
    max_length=MAX_LENGTH,
    tokenizer_name=TOKENIZER_NAME,
    use_prefix=USE_PREFIX
)
dev_data_loader = DataLoader(
    dev_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=False
)
print("The number of validation batches: ", len(dev_data_loader))
label_size = len(train_dataset.label_names)

test_dataset = DataClass(
    filename=TEST_FILEPATH,
    max_length=MAX_LENGTH,
    tokenizer_name=TOKENIZER_NAME,
    use_prefix=USE_PREFIX
)
test_data_loader = DataLoader(
    test_dataset, batch_size=EVAL_BATCH_SIZE, shuffle=False
)
print("The number of Test batches: ", len(test_data_loader))

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

The number of training batches:  120
Reading corpus_id_web_wiki - 1grams ...
You can't omit/backoff and unpack hashtags!
 unpack_hashtags will be set to False


PreProcessing dataset ...: 100%|██████████| 525/525 [00:00<00:00, 782.21it/s]
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


The number of validation batches:  33
Reading corpus_id_web_wiki - 1grams ...
You can't omit/backoff and unpack hashtags!
 unpack_hashtags will be set to False


PreProcessing dataset ...: 100%|██████████| 1079/1079 [00:01<00:00, 752.32it/s]


The number of Test batches:  68


# Training

In [10]:
#############################################################################
# Start Training
#############################################################################



test_results = {}
eval_results = {}

for lca, zlpr in zip(LCA_LOSSES, ZLPR_LOSSES):
    
    set_seed(seed)
    
    spanemo = SpanEmo(
        output_dropout=DROPOUT_RATE,
        lang="Indonesia",
        alpha=lca,
        beta=zlpr,
        embedding_vocab_size=len(train_dataset.tokenizer),
        device=device,
        encoder_name=ENCODER_NAME,
    )

    emorec = EmoRec(
        output_dropout=DROPOUT_RATE,
        lang="Indonesia",
        alpha=lca,
        beta=zlpr,
        embedding_vocab_size=len(train_dataset.tokenizer),
        label_size=len(train_dataset.label_names),
        device=device,
        encoder_name=ENCODER_NAME,
    )
    
    models = [spanemo, emorec]
    
    model = [x for x in models if x.name == MODEL_TO_TRAIN][0]

    run_name = f"{model.name}-roberta-BCE{str(1-lca-zlpr).replace('.','x')}-ZLPR{str(zlpr).replace('.','x')}-LCA{str(lca).replace('.','x')}"
    
    early_stopping = EarlyStopping(filename = filename, patience=PATIENCE, delta=0, criteria="f1_Macro", bigger_better=True)
    
    learn = Trainer(
        model,
        train_data_loader,
        dev_data_loader,
        filename=filename,
        col_names=train_dataset.label_names,
        early_stopping = early_stopping
    )
    
    learn.fit(
        classifier_learning_rate = CLASSIFIER_LEARNING_RATE,
        encoder_learning_rate = ENCODER_LEARNING_RATE,
        train_batch_size = TRAIN_BATCH_SIZE,
        max_epoch = MAX_EPOCH,
        project_name = PROJECT_NAME,
        run_name = run_name,
        device=device
    )
    
    evaluate = EvaluateOnTest(
        model, 
        test_data_loader, 
        model_path=MODEL_PATH, 
        col_names=train_dataset.label_names,
        run_name = "_test_"+run_name
    )
    evaluate_dev = EvaluateOnTest(
        model, 
        dev_data_loader, 
        model_path=MODEL_PATH, 
        col_names=train_dataset.label_names,
        run_name = "_dev_"+run_name
    )
    print("==========",run_name, "===========")
    
    test_results[run_name] = evaluate.predict(device=device)
    eval_results[run_name] = evaluate_dev.predict(device=device)
    
    wandb.finish()

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
wandb: Currently logged in as: syafiqfaray3. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Train_Loss,Val_Loss,F1-Macro,F1-Micro,Precision Macro,Recall Macro,JS-Macro,JS-Samples,MCC,Time
1,0.3951,0.3215,0.2645,0.5614,0.8688,0.2567,0.1991,0.4073,0.5029,02:34
2,0.3025,0.3167,0.4271,0.6744,0.7732,0.4461,0.3218,0.5514,0.5998,02:39
3,0.2397,0.2890,0.4956,0.7016,0.7116,0.4716,0.3790,0.6111,0.6408,02:40
4,0.1740,0.3467,0.5690,0.7070,0.6250,0.6450,0.4274,0.6043,0.6388,02:40
5,0.1158,0.3184,0.6150,0.7133,0.6522,0.5971,0.4651,0.6243,0.6493,02:40
6,0.0807,0.4377,0.6185,0.7064,0.6413,0.6555,0.4618,0.6081,0.6368,02:40
7,0.0580,0.4993,0.5837,0.7109,0.6869,0.5544,0.4331,0.6097,0.6455,02:40
8,0.0361,0.6249,0.6312,0.7190,0.6484,0.6320,0.4767,0.6351,0.6551,02:40
9,0.0324,0.6180,0.6336,0.7265,0.6526,0.6279,0.4776,0.6490,0.6650,02:40
10,0.0139,0.7812,0.6332,0.7274,0.6539,0.6229,0.4790,0.6390,0.6655,02:40


...

Epoch,Train_Loss,Val_Loss,F1-Macro,F1-Micro,Precision Macro,Recall Macro,JS-Macro,JS-Samples,MCC,Time
1,1.3626,1.1938,0.3975,0.6090,0.7094,0.3906,0.2874,0.4835,0.5276,02:40
2,1.1425,1.1860,0.3940,0.6603,0.8219,0.4142,0.2998,0.5238,0.5834,02:40
3,0.9515,1.1502,0.5017,0.7060,0.5792,0.4748,0.3819,0.6184,0.6431,02:40
4,0.7403,1.3489,0.5903,0.7028,0.6248,0.6303,0.4411,0.6046,0.6325,02:40
5,0.5891,1.1943,0.6206,0.7425,0.6648,0.6097,0.4754,0.6606,0.6836,02:40
6,0.4604,1.5433,0.6306,0.7232,0.6520,0.6370,0.4758,0.6340,0.6583,02:40
7,0.3893,1.8036,0.5838,0.7187,0.7267,0.5390,0.4334,0.6108,0.6565,02:40
8,0.3304,2.2791,0.6160,0.7226,0.6766,0.5899,0.4632,0.6356,0.6602,02:40
9,0.3171,2.5370,0.6571,0.7297,0.6391,0.6850,0.5004,0.6537,0.6664,02:40
10,0.2823,2.8263,0.6303,0.7277,0.6870,0.6025,0.4772,0.6473,0.6675,02:40


epoch#:  1
f1_Macro increased (inf --> 0.397491).  Saving model ...
epoch#:  2
EarlyStopping counter: 1 out of 20
epoch#:  3
f1_Macro increased (0.397491 --> 0.501696).  Saving model ...
epoch#:  4
f1_Macro increased (0.501696 --> 0.590312).  Saving model ...
epoch#:  5
f1_Macro increased (0.590312 --> 0.620592).  Saving model ...
epoch#:  6
f1_Macro increased (0.620592 --> 0.630581).  Saving model ...
epoch#:  7
EarlyStopping counter: 1 out of 20
epoch#:  8
EarlyStopping counter: 2 out of 20
epoch#:  9
f1_Macro increased (0.630581 --> 0.657106).  Saving model ...
epoch#:  10
EarlyStopping counter: 1 out of 20
epoch#:  11
EarlyStopping counter: 2 out of 20
epoch#:  12
EarlyStopping counter: 3 out of 20
epoch#:  13
EarlyStopping counter: 4 out of 20
epoch#:  14
EarlyStopping counter: 5 out of 20
epoch#:  15
f1_Macro increased (0.657106 --> 0.658791).  Saving model ...
epoch#:  16
EarlyStopping counter: 1 out of 20
epoch#:  17
EarlyStopping counter: 2 out of 20
epoch#:  18
EarlyStopping 

F1-Macro: 0.6532 F1-Micro: 0.7410 Precision Macro: 0.7151 Recall Macro: 0.6167 JS-Macro: 0.4964 JS-Samples: 0.6574 MCC: 0.6828 Time: 00:28
Confusion Matrix:
[[[ 541   82]
  [  69  387]]

 [[ 979   16]
  [  43   41]]

 [[1033    6]
  [  24   16]]

 [[ 608  110]
  [  73  288]]

 [[ 857   50]
  [  47  125]]

 [[1023   12]
  [  25   19]]

 [[ 767   70]
  [  91  151]]]


F1-Macro: 0.6588 F1-Micro: 0.7465 Precision Macro: 0.7276 Recall Macro: 0.6214 JS-Macro: 0.5074 JS-Samples: 0.6644 MCC: 0.6897 Time: 00:13
Confusion Matrix:
[[[256  41]
  [ 29 199]]

 [[477   8]
  [ 24  16]]

 [[501   3]
  [ 13   8]]

 [[299  50]
  [ 39 137]]

 [[432  12]
  [ 16  65]]

 [[494   5]
  [ 13  13]]

 [[373  48]
  [ 40  64]]]


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
f1_Macro,▁▁▄▆▇▇▆▇█▇▇▇████████
f1_Micro,▁▄▆▆█▇▇▇▇▇▇▇█▇█▇████
js_Macro,▁▁▄▆▇▇▆▇█▇▇▇█▇█▇████
js_Samples,▁▃▆▆█▇▆▇█▇▇▇█▇████▇▇
mcc,▁▃▆▆█▇▇▇▇▇▇▇█▇█▇████
precision_macro,▅█▁▂▃▃▅▄▃▄▄▅▅▄▅▄▅▅▄▄
recall_macro,▁▂▃▇▆▇▅▆█▆▆▆▆▇▆▆▆▇▇▆
train_loss,█▇▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
val_loss,▁▁▁▁▁▂▃▄▄▅▆▇▇▇▇█████
epoch,20


In [11]:
import pickle 
with open(f'{model.name}_test.pkl', 'wb') as f:
    pickle.dump(test_results, f)
    
with open(f'{model.name}_eval.pkl', 'wb') as f:
    pickle.dump(eval_results, f)

In [12]:
evaluate = EvaluateOnTest(
        model, 
        test_data_loader, 
        model_path=MODEL_PATH, 
        col_names=train_dataset.label_names
    )
evaluate.predict(device=device)

F1-Macro: 0.6532 F1-Micro: 0.7410 Precision Macro: 0.7151 Recall Macro: 0.6167 JS-Macro: 0.4964 JS-Samples: 0.6574 MCC: 0.6828 Time: 00:28
Confusion Matrix:
[[[ 541   82]
  [  69  387]]

 [[ 979   16]
  [  43   41]]

 [[1033    6]
  [  24   16]]

 [[ 608  110]
  [  73  288]]

 [[ 857   50]
  [  47  125]]

 [[1023   12]
  [  25   19]]

 [[ 767   70]
  [  91  151]]]


{'f1-macro': 0.6532478476681745,
 'f1-micro': 0.7409812409812411,
 'precision-macro': 0.7151136909564828,
 'recall-macro': 0.6167275274504304,
 'js-macro': 0.4964207209215874,
 'js-samples': 0.657398826073525,
 'mcc': 0.6828194856643677}